In [ ]:
# -*-coding:utf8-*-#
__author__ = 'play4fun'
"""
create time:15-10-24 下午5:46
原理
模板匹配是用来在一副大图中搜寻查找模版图像位置的方法。OpenCV 为 我们提供了函数 cv2.matchTemplate()。
和 2D 卷积一样 它也是用模板图像在输入图像 大图 上滑动 并在每一个位置对模板图像和与其对应的 输入图像的子区域  比较。
OpenCV 提供了几种不同的比较方法 细节 看 文档 。 
返回的结果是一个灰度图像 每一个像素值 示了此区域与模板的匹配 程度。
如果输入图像的大小是 WxH  
模板的大小是 wxh   输出的结果 的大小就是 W-w+1 H-h+1 。
当你得到这幅图之后 就可以使用函数 cv2.minMaxLoc() 来找到其中的最小值和最大值的位置了。
第一个值为矩形左上角的点 位置 
w h 为 moban 模板矩形的宽和 。
这个矩形就是 找到的模板区域了。
"""

import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('../data/messi5.jpg', 0)
img2 = img.copy()
template = cv2.imread('../data/messi_face.jpg', 0)

w, h = template.shape[::-1]
# All the 6 methods for comparison in a list
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
           'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

for meth in methods:
    img = img2.copy()

    # exec 语句用来执行储存在字符串或文件中的 Python 语句。
    # 例如,我们可以在运行时生成一个包含 Python 代码的字符串,
    # 然后使用 exec 语句执行这些语句。
    # eval 语句用来计算存储在字符串中的有效 Python 表达式
    method = eval(meth)
    # Apply template Matching
    res = cv2.matchTemplate(img, template, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    # 使用不同的比较方法,对结果的解释不同
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc

    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img, top_left, bottom_right, 255, 2)

    plt.subplot(121), plt.imshow(res, cmap='gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(img, cmap='gray')
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.suptitle('method: ' + meth)
    plt.show()


In [ ]:
'''
假如你的目标对 只在图像中出现了很多次怎么办呢
函数 cv.imMaxLoc() 只会给出最大值和最小值。此时 我们就 使用阈值了。
在下 的例子中我们 经典游戏 Mario 的一张截屏图片中找到其中的硬币

'''

import cv2
import numpy as np

# from matplotlib import pyplot as plt

img_rgb = cv2.imread('../data/mario.png')
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('../data/mario_coin.png', 0)
w, h = template.shape[::-1]

res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where(res >= threshold)
print(len(loc))

for pt in zip(*loc[::-1]):
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
    print("rectangle 1")

# cv2.imwrite('res.png',img_rgb)
cv2.imshow("result", img_rgb)
cv2.waitKey(0)


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/26 23:42
# @Author  : play4fun
# @File    : LineSegmentDetector1.py
# @Software: PyCharm

"""
LineSegmentDetector1.py:
"""
import cv2
import numpy as np

# Read gray image
img0 = cv2.imread("pokerQ.jpg")
img = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)
cv2.imshow('pokerQ',img0)

# Create default parametrization LSD
lsd = cv2.createLineSegmentDetector(0)

# Detect lines in the image
dlines = lsd.detect(img)#TODO 返回什么？
lines = lsd.detect(img)[0]  # Position 0 of the returned tuple are the detected lines

# Draw detected lines in the image
# drawn_img = lsd.drawSegments(img, lines)

#
cv2.waitKey(0)
for dline in dlines[0]:
    x0 = int(round(dline[0][0]))
    y0 = int(round(dline[0][1]))
    x1 = int(round(dline[0][2]))
    y1 = int(round(dline[0][3]))
    cv2.line(img0, (x0, y0), (x1,y1), (0,255,0), 1, cv2.LINE_AA)
    cv2.imshow("LSD", img0)
    cv2.waitKey(200)

#TODO 最长的直线？

# Show image
# cv2.imshow("LSD", drawn_img)
# cv2.imshow("LSD", img0)
# cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/9 下午12:01
# @Author  : play4fun
# @File    : HoughLinesP_camera.py
# @Software: PyCharm

"""
HoughLinesP_camera.py:
"""

import cv2
import numpy as np

cap = cv2.VideoCapture(0)
# ret = cap.set(3, 640)
# ret = cap.set(4, 480)

# while (True):
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    # img = cv2.imread('../data/sudoku.jpg')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength, maxLineGap)
    if lines is None:
        continue
    print("Len of lines:", len(lines))
    print(lines)

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # cv2.imwrite('houghlines5.jpg',img)
    cv2.imshow("houghlines3.jpg", frame)

    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


In [ ]:
# -*-coding:utf8-*-#
__author__ = 'play4fun'
"""
create time:15-10-25 上午11:42

cv2.HoughLines()。 返回值就是 ρ, θ 。
ρ 的单位是像素  θ 的单位是弧度。 
第一个参数是一个二值化图像 所以在进行霍夫变换之前  先进行  二值化 或者  
Canny  缘检测。
第二和第三个值分别代  ρ 和 θ 的精确度。
第四个参数是阈值， 只有累加其中的值高于阈值时才被认为是一条直线 
也可以把它看成能检测到的直线的最短长度 以像素点为单位 。
"""

import cv2
import numpy as np

img = cv2.imread('../data/sudoku.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# edges = cv2.Canny(gray,50,150,apertureSize = 3)
edges = cv2.Canny(gray, 10, 50, apertureSize=3)
cv2.imshow("edges", edges)

lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
print("Len of lines:", len(lines))
# print lines

for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # cv2.imwrite('houghlines3.jpg',img)
    cv2.imshow("houghlines3.jpg", img)
    cv2.waitKey(1000)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# -*-coding:utf8-*-#
__author__ = 'play4fun'
"""
create time:15-10-25 上午11:53
"""

import cv2
import numpy as np

img = cv2.imread('../data/sudoku.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

minLineLength = 100
maxLineGap = 10

lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength, maxLineGap)
print("Len of lines:", len(lines))
print(lines)

for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

# cv2.imwrite('houghlines5.jpg',img)
cv2.imshow("houghlines3.jpg", img)
cv2.waitKey(0)


In [ ]:
#!/usr/bin/python

# Standard imports
import cv2
import numpy as np;

# Read image
im = cv2.imread("blob.jpg", cv2.IMREAD_GRAYSCALE)

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 10
params.maxThreshold = 200

# Filter by Area.
params.filterByArea = True
params.minArea = 1500

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.87

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

# Create a detector with the parameters
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3:
    detector = cv2.SimpleBlobDetector(params)
else:
    detector = cv2.SimpleBlobDetector_create(params)

# Detect blobs.
keypoints = detector.detect(im)

# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
# the size of the circle corresponds to the size of blob

im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show blobs
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/24 下午4:51
# @Author  : play4fun
# @File    : 斑点检测SimpleBlobDetector.py
# @Software: PyCharm

"""
斑点检测SimpleBlobDetector.py:
https://www.learnopencv.com/blob-detection-using-opencv-python-c/

特别要注意，默认检测黑色点，如果要检测白色的点请设置bycolor为true，并且color数值是255.


斑点通常是指与周围有着颜色和灰度差别的区域。在实际地图中，往往存在着大量这样的斑点，如一颗树是一个斑点，一块草地是一个斑点，一栋房子也可以是一个斑点。由于斑点代表的是一个区域，相比单纯的角点，它的稳定性要好，抗噪声能力要强，所以它在图像配准上扮演了很重要的角色。

同时有时图像中的斑点也是我们关心的区域，比如在医学与生物领域，我们需要从一些X光照片或细胞显微照片中提取一些具有特殊意义的斑点的位置或数量。

比如下图中天空的飞机、向日葵的花盘、X线断层图像中的两个斑点。

"""

# Standard imports
import cv2
import numpy as np

# Read image
im = cv2.imread("blob.jpg", cv2.IMREAD_GRAYSCALE)
# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create()
# Detect blobs.
keypoints = detector.detect(im)
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# Show keypoints
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/24 下午5:03
# @Author  : play4fun
# @File    : blob_camera.py
# @Software: PyCharm

"""
blob_camera.py:
"""

import cv2
import numpy as np

# Read image
# im = cv2.imread("blob.jpg", cv2.IMREAD_GRAYSCALE)
# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create()

cap = cv2.VideoCapture(0)
while cap.isOpened():  # 检查是否成功初始化，否则就 使用函数 cap.open()
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect blobs.
    keypoints = detector.detect(gray)
    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Show keypoints
    cv2.imshow("Keypoints", im_with_keypoints)

    key = cv2.waitKey(delay=1)
    if key == ord("q"):
        break

cv2.destroyAllWindows()


In [ ]:
# -*-coding:utf8-*-#
__author__ = 'play4fun'
"""
create time:15-10-25 下午12:02
一个圆环 需要 3 个参数来确定。所以进行圆环 夫变换的累加器必须是 3 维的
  这样的 效率 就会很低。所以 OpenCV 用来一个比 巧妙的办法 霍夫梯度法 它可以使 用边界的梯度信息。
  
参数：
image： 8位，单通道图像。如果使用彩色图像，请先转换为灰度。
method：定义检测图像中的圆的方法。目前，唯一实现的方法是cv2.HOUGH_GRADIENT对应于Yuen等。纸。
dp：该参数是累加器分辨率与图像分辨率的反比（详见Yuen等人）。实质上，dp获取越大，累加器数组越小。
minDist：检测到的圆的中心（x，y）坐标之间的最小距离。如果minDist太小，则可能（错误地）检测到与原始相邻的多个圆。如果minDist太大，那么一些圈子根本就不会被检测到。
param1： Yuen等人用于处理边缘检测的梯度值 方法。
param2：该cv2.HOUGH_GRADIENT方法的累加器阈值。阈值越小，检测到的圈子越多（包括虚假圈子）。阈值越大，可能会返回的圈数越多。
minRadius：半径的最小大小（以像素为单位）。
maxRadius：半径的最大大小（以像素为单位）。
"""

import cv2
import numpy as np

img = cv2.imread('img/OpenCV_Logo_with_text.png', 0)
img = cv2.medianBlur(img, 5)
cimg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

#HoughCircles(image, method, dp, minDist, circles=None, param1=None, param2=None, minRadius=None, maxRadius=None)
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

circles = np.uint16(np.around(circles))
print(circles)

for i in circles[0, :]:
    # draw the outer circle
    cv2.circle(cimg, (i[0], i[1]), i[2], (0, 255, 0), 2)
    # draw the center of the circle
    cv2.circle(cimg, (i[0], i[1]), 2, (0, 0, 255), 3)

cv2.imshow('detected circles', cimg)
cv2.waitKey(10000)
cv2.destroyAllWindows()

# Python: cv2.HoughCircles(image, method, dp, minDist, circles, param1, param2, minRadius, maxRadius)
# Parameters:
# image – 8-bit, single-channel, grayscale input image.
# 返回结果为 Output vector of found circles. Each vector is encoded as a
# 3-element floating-point vector (x, y, radius) .
# circle_storage – In C function this is a memory storage that will contain
# the output sequence of found circles.
# method – Detection method to use. Currently, the only implemented method is
# CV_HOUGH_GRADIENT , which is basically 21HT , described in [Yuen90].
# dp – Inverse ratio of the accumulator resolution to the image resolution.
# For example, if dp=1 , the accumulator has the same resolution as the input image.
# If dp=2 , the accumulator has half as big width and height.
# minDist – Minimum distance between the centers of the detected circles.
# If the parameter is too small, multiple neighbor circles may be falsely
# detected in addition to a true one. If it is too large, some circles may be missed.
# param1 – First method-specific parameter. In case of CV_HOUGH_GRADIENT ,
# it is the higher threshold of the two passed to the Canny() edge detector
# (the lower one is twice smaller).
# param2 – Second method-specific parameter. In case of CV_HOUGH_GRADIENT ,
# it is the accumulator threshold for the circle centers at the detection stage.
# The smaller it is, the more false circles may be detected. Circles,
# corresponding to the larger accumulator values, will be returned first.
# minRadius – Minimum circle radius.
# maxRadius – Maximum circle radius.


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/27 13:47
# @Author  : play4fun
# @File    : HoughCircles_camera.py
# @Software: PyCharm

"""
HoughCircles_camera.py:

用围棋-棋子来测试
"""

import cv2
import numpy as np
from skimage.measure import compare_mse as mse
import string, random


def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))


cap = cv2.VideoCapture(0)

# ret = cap.set(3, 640)
# ret = cap.set(4, 480)

# margin = 60
margin = 30


def draw_line_rectangle(frame, margin):
    rows, cols, ch = frame.shape  # (720, 1280, 3)
    half = int(cols / 2)
    # 中间
    cv2.line(frame, (half, 0), (half, rows), (0, 0, 255), 2)

    # margin = 40
    # 左边
    up_left1 = (margin, margin)  # 左上点
    down_right1 = (cols - margin, rows - margin)  # 右下点
    # print(up_left, down_right)
    cv2.rectangle(frame, up_left1, down_right1, (0, 255, 0), 3)


ret, temp = cap.read()
tm = 0
while cap.isOpened():
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
    if key == ord('s'):
        cv2.imwrite(id_generator() + '.jpg', frame2)

    # Capture frame-by-frame
    ret, frame = cap.read()
    m = mse(cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY), cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    print('mse', m, '----\n')
    if abs(m - tm) < 2:  # 静止画面，不用重复计算
        continue
    else:
        temp = frame.copy()
        tm = m
    #
    # print(margin,frame.shape[0] - margin, margin,frame.shape[1] - margin)#40 680 40 1240
    frame2 = frame[margin:frame.shape[0] - margin, margin:frame.shape[1] - margin]  # .copy()
    # cv2.imshow('frame2', frame2)

    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # HoughCircles(image, method, dp, minDist, circles=None, param1=None, param2=None, minRadius=None, maxRadius=None)
    # circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=100, param2=30, minRadius=10, maxRadius=40)

    # circles = circles1[0, :, :]  # 提取为二维
    # circles = np.uint16(np.around(circles1))
    print(circles)

    cimg = frame2
    if circles is not None:
        for i in circles[0, :]:
            # for i in circles[:]:
            # draw the outer circle
            cv2.circle(cimg, (i[0], i[1]), i[2], (0, 255, 0), 2)
            # draw the center of the circle
            cv2.circle(cimg, (i[0], i[1]), 2, (0, 0, 255), 3)

    # cv2.imshow('detected circles', cimg)

    draw_line_rectangle(frame, margin)
    cv2.imshow("houghlines", frame)
    # cv2.imwrite('frame3.jpg', frame[margin:frame.shape[0] - margin, margin:frame.shape[1] - margin])

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/28 11:05
# @Author  : play4fun
# @File    : HoughCircles_chess.py
# @Software: PyCharm

"""
HoughCircles_chess.py:
围棋
"""

import cv2
import numpy as np
from collections import Counter


def detect_weiqi(img):  # 检测棋子的颜色
    txt = 'black'
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

    c = Counter(list(threshold.flatten()))
    print(c.most_common())
    if c.most_common()[0][0] != 0:
        txt = 'white'
    return txt, threshold


img = cv2.imread('img/weiqi.png')

img = cv2.medianBlur(img, 5)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# ret, threshold = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)#不行
cv2.imshow('gray', gray)
# cv2.imshow('threshold', threshold)
# cv2.waitKey(0)

# HoughCircles(image, method, dp, minDist, circles=None, param1=None, param2=None, minRadius=None, maxRadius=None)
# circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=10, maxRadius=40)#有一些没有检测到
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=100, param2=30, minRadius=10, maxRadius=50)

if circles is None:
    exit(-1)

circles = np.uint16(np.around(circles))
print(circles)

cv2.waitKey(0)
font = cv2.FONT_HERSHEY_SIMPLEX
for i in circles[0, :]:
    # draw the outer circle
    cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
    # draw the center of the circle
    cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)

    x, y, r = i
    crop_img = img[y - r:y + r, x - r:x + r]
    # 检测围棋
    txt, threshold = detect_weiqi(crop_img)
    print('颜色：', '黑色' if txt == 'black' else '白色')

    cv2.putText(threshold, text=txt, org=(0, 0), fontFace=font, fontScale=0.5, color=(0, 255, 0), thickness=2)
    cv2.imshow('threshold', threshold)

    cv2.imshow('crop_img', crop_img)
    cv2.moveWindow('crop_img', x=0, y=img.shape[0])

    cv2.imshow('detected chess', img)
    cv2.moveWindow('detected chess', y=0, x=img.shape[1])

    cv2.waitKey(1500)

cv2.waitKey(10000)
cv2.destroyAllWindows()


[[[296 258  28]
  [360 338  29]
  [302 138  28]
  [356 272  29]
  [240 274  29]
  [176 262  28]
  [256 330  29]
  [176 132  28]
  [426 272  28]
  [240 136  29]
  [308  78  28]
  [264 388  29]
  [242 200  28]
  [180 204  28]
  [ 84 132  27]
  [324 382  29]
  [368 148  28]
  [250  70  29]]]


In [2]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/27 10:57
# @Author  : play4fun
# @File    : HoughCircles_eyes.py
# @Software: PyCharm

"""
HoughCircles_eyes.py:

http://blog.csdn.net/on2way/article/details/47028969

"""
%matplotlib
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('eye-color-blue-z-c-660x440.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 灰度图像

plt.subplot(121), plt.imshow(gray, 'gray')
plt.xticks([]), plt.yticks([])
# hough transform   #规定检测的圆的最大最小半径，不能盲目的检测，否侧浪费时间空间。
# circles1 = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1,100, param1=100, param2=30, minRadius=200, maxRadius=300)
circles1 = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=30, minRadius=100, maxRadius=200)  # 把半径范围调小点，检测内圆,瞳孔
circles = circles1[0, :, :]  # 提取为二维
circles = np.uint16(np.around(circles))  # 四舍五入，取整
for i in circles[:]:
    cv2.circle(img, (i[0], i[1]), i[2], (255, 0, 0), 5)  # 画圆
    cv2.circle(img, (i[0], i[1]), 2, (255, 0, 255), 10)  # 画圆心

plt.subplot(122), plt.imshow(img)
plt.xticks([]), plt.yticks([])
plt.show()


Using matplotlib backend: Qt5Agg
